In [1]:
import json
from coincurve import PrivateKey, PublicKey
from hashlib import sha256

def hash_to_curve(x_bytes):
    # Hash the secret using SHA-256
    hash_value = sha256(x_bytes).digest()
    # Create a public key Y from the hashed secret
    Y = PublicKey.from_secret(hash_value)
    return Y

def subtract_points(pt1: PublicKey, pt2:  PublicKey):
    p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
    x2, y2 = pt2.point()
    neg_pt2 = PublicKey.from_point(x2, (p - y2) % p)
    difference = PublicKey.combine_keys([pt1, neg_pt2])
    return difference


In [5]:
l1 = "/usr/local/bin/lightning-cli --lightning-dir=/tmp/l1"
l2 = "/usr/local/bin/lightning-cli --lightning-dir=/tmp/l2"

# mint 7 ecash tokens in exchange for 7 sats
mint_amount = 7

# melt 7 tokens for 7 sats
melt_amount = 7

# Define Keyset

In [8]:
 !$l1 cashu-get-keys

{
   "code": -32601,
   "message": "Unknown command 'cashu-get-keys'"
}


In [9]:
pubkeys = !$l1 cashu-get-keys | jq -r .keysets[0].keys
pubkeys = json.loads(''.join(pubkeys))
pubkeys

{'1': '02b53a33e8b70644d83e5700b2dfa45b7fd7bb1ed597a6f4f2408e79d2ed316cef',
 '2': '034c307cfa8736158e2bb0435153f805e7d6d6ec95249eee32f31effe4fef87e24',
 '4': '02837b0c27e0e1c00934925031e10c41e7a2b926eb1b6e8d5846024b64b16964cc',
 '8': '0232eaf6f41725360b28ba5242243616099dce2f17aaf5fa7241bb4015d6461495',
 '16': '031b06e936ace0e0e449a5eeabcb0dc39a11b72650142cc653d008dce482004178',
 '32': '03053d1a6e05e1e12537129dad7c983cc1845279f3051df7d37956cdb09d633f29',
 '64': '02f81334344f31fe0b919bd9fa40e4a5b0b4e4d4dbd017cdf6dbe66d1fbfae1975',
 '128': '02ee1a9c0b7018150ff71ce3f2207011f034a6a5a81d40f2066105254993b4b305'}

In [10]:
privkeys = !$l1 cashu-dev-get-privkeys
privkeys = json.loads(''.join(privkeys))
privkeys

{'1': '78196135e2121c6750e0d3ed7669cc43a83107d79dbfbb253c069292ec83c59a',
 '2': '62d469a41ab8e82cb95dce66a8562f8a241d9f8c1c9dbaac41829d46d1856bb7',
 '4': '3375381bb41a9cb1b1a47d78d37e4a703332af50ef4d1661a8892ba3f0e42f8c',
 '8': '5c47dc806dac67518dd8355dc1c2b29daf73a88195913b5b1b8e5bf319d40ae7',
 '16': '2f8cae4671fadfec38df2d8afa2b57b5303c5035488e23d373b2bb96139b9992',
 '32': '226d7f6783a73927ece9009124589aaf107ea0edafb9f4c7c5358301b54e7888',
 '64': '52e4b9328d26a927e0473e806f52fa058027c649f9bfee45f8ed13c585892fcd',
 '128': '2207ce69cd468de1790ad1c8a5c9516ac6f6068067bf00b1e4545e134bb70fb8'}

# Minting and Melting Cashu Tokens

The following describes the path from:

    1. lightning --> ecash (**minting**)
    2. ecash --> lightning (**melting**)

## Minting
A wallet pays a lightning invoice in exchange for signatures from a mint on blinded messages.

### 1. Wallet requests a quote to *mint*

In [91]:
mint_quote = !$l1 -k cashu-quote-mint amount=$mint_amount unit="sat" | jq -r 
mint_quote = json.loads(''.join(mint_quote))

# quote is the unique id for this exchange
mint_id = mint_quote["quote"]
# `request` is a lightning invoice
mint_invoice = mint_quote["request"]

print("quoteId:", mint_id)
print("bolt11: ", mint_invoice)

quoteId: 7c6e4dc73f7bbf0f
bolt11:  lnbcrt70n1pje7tqssp5cahdxfe3s83evhv48e29jv8qqp0wrpna0yl4g98ff23lzzme9wxqpp5p9e207sy46ry46wruxll0frqrerzcu6ljl3c874xjdmc020aymcqdqsg9hzq6twwehkjcm9xqyjw5qcqp29qx3qysgqscuaenee3mqqft6warm6z0w67cu7u257jhr96ydsa5k55wrxvz0r33skmc2d5x068mj0dfk758lmh0tzgfztxz4xd9vffvjlqcsettsp7x9aqr


### 2. Wallet pays invoice

In [92]:
!$l2 pay "$mint_invoice" | jq -r .status

complete


### 3. Check mint status

In [93]:
!$l1 cashu-check-mint "$mint_id"

{
   "quote": "7c6e4dc73f7bbf0f",
   "request": "lnbcrt70n1pje7tqssp5cahdxfe3s83evhv48e29jv8qqp0wrpna0yl4g98ff23lzzme9wxqpp5p9e207sy46ry46wruxll0frqrerzcu6ljl3c874xjdmc020aymcqdqsg9hzq6twwehkjcm9xqyjw5qcqp29qx3qysgqscuaenee3mqqft6warm6z0w67cu7u257jhr96ydsa5k55wrxvz0r33skmc2d5x068mj0dfk758lmh0tzgfztxz4xd9vffvjlqcsettsp7x9aqr",
   "paid": true,
   "expiry": 1705535120
}


### 4. Wallet requests tokens from mint
- wallet must first generate blinded messages from tokens they want

In [94]:
def generate_blinded_messages(secrets, amounts):
    assert len(secrets) == len(amounts)
    BlindedMessages =  []
    rs = []
    for s, amount in zip(secrets, amounts):
        # r is a random blinding factor
        r = PrivateKey()
        R = r.public_key
        #Y = hash_to_curve(x)
        Y = hash_to_curve(s.encode())
        # B_ = Y + rG
        B_ = PublicKey.combine_keys([Y, R]).format().hex()
        BlindedMessages.append({"amount": amount, "B_": B_})
        rs.append(r.secret)
    return BlindedMessages, rs

In [95]:
secrets = ['1', '2', '4']
amounts = [1, 2, 4]
b_messages, rs = generate_blinded_messages(secrets, amounts)

b_messages = json.dumps(b_messages)
b_messages


'[{"amount": 1, "B_": "02c4461234b63936a3e3c9dea331a73cd16c1fb1a4084a0841dc7aaae6b64d15e5"}, {"amount": 2, "B_": "025d89874048a5f10878d8f0bfd7eb25947f9012149009aee0bc3bb897e76eec74"}, {"amount": 4, "B_": "02e91f15ad2c3fc4ad9be5421701c8674baa6202f6bb3b65a7e37f3eaab2101170"}]'

#### 4b. wallet sends blinded msgs for  blinded sigs

[
   {
      "amount": 1,
      "id": "00b2f181c83b11aa",
      "C_": "0208f205d3bbc2e17b62d58ed3fc873c788b86ae3e48da94d07a77fed586409e9e"
   },
   {
      "amount": 2,
      "id": "00b2f181c83b11aa",
      "C_": "023a7623efdd39dafd097c866e25956b2d24f91fcdb1f66803059ea1ea0c348038"
   },
   {
      "amount": 4,
      "id": "00b2f181c83b11aa",
      "C_": "026bed9f443ab7342d69035a89373b78c11f363b939708da76c738340585289a6a"
   }
]


In [96]:
blinded_sigs = ! /usr/local/bin/lightning-cli --lightning-dir=/tmp/l1 -k cashu-mint quote=7c6e4dc73f7bbf0f blinded_messages='[{"amount": 1, "B_": "02c4461234b63936a3e3c9dea331a73cd16c1fb1a4084a0841dc7aaae6b64d15e5"}, {"amount": 2, "B_": "025d89874048a5f10878d8f0bfd7eb25947f9012149009aee0bc3bb897e76eec74"}, {"amount": 4, "B_": "02e91f15ad2c3fc4ad9be5421701c8674baa6202f6bb3b65a7e37f3eaab2101170"}]'
blinded_sigs = json.loads(''.join(blinded_sigs))
blinded_sigs

[{'amount': 1,
  'id': '00b2f181c83b11aa',
  'C_': '02b05219745a74d9dffa94e0d2a085cfb735dac70b5704d84a7d517e86e88374d0'},
 {'amount': 2,
  'id': '00b2f181c83b11aa',
  'C_': '0265943297e57845c6ee38bd5560a5051b428ca8401180258f1404e955477eee5b'},
 {'amount': 4,
  'id': '00b2f181c83b11aa',
  'C_': '03431f257071037ebe8fda5993e7b712e8d915f23b92147d8ea3754c2eb23d31bc'}]

## Melting

### 5. Wallet generates and invoice for mint to pay
- in this case `l2` will be the wallet

In [109]:
melt_invoice = ! /usr/local/bin/lightning-cli --lightning-dir=/tmp/l2 invoice 7000 $RANDOM description  | jq -r .'bolt11'
melt_invoice = melt_invoice[0]
melt_invoice

'lnbcrt70n1pje7t66sp5jpve97n3nshujzfnxf4ax5u3eed7htc7ec5pqjpgdn4pv4vflukspp5gu52wmsd5lw9wvxf3ce36r8kl4vhwjm2x887escdq3n0thavh4fqdqjv3jhxcmjd9c8g6t0dcxqyjw5qcqp29qx3qysgqetvu9qn0dg9hqgxqh6k37wx9f4lyfrf59mwpuzq92vm05eaysjgrg4jg4z468nvhc53er83a54la2afymdd7vvfdtrqj5p7r0uq2lcqqv7mmxy'

### 6. Wallet requests a quote to *melt*

In [112]:
# the amount_msat in the first hop should be total fee + final amount
# get fee and then maybe +20%
!$l1 -k cashu-quote-melt req=$melt_invoice unit="sat"

{
   "route": [
      {
         "id": "020508bfd8c5134b5377342d97b85bb3f7dd8a26ed3f3297362b033d79c18f7b61",
         "channel": "609x1x1",
         "direction": 1,
         "amount_msat": "7000msat",
         "delay": 9,
         "style": "tlv"
      }
   ]
}


In [108]:
melt_quote = !$l1 -k cashu-quote-melt req=$melt_invoice unit="sat" | jq -r .'quote'
melt_quote = melt_quote[0]
melt_quote

'null'

### 7. Wallet sends tokens for the mint to melt

In [99]:
def verify_token(C, secret_bytes, k: str):
    # k*hash_to_curve(x) == C
    Y = hash_to_curve(secret_bytes)
    kY = Y.multiply(bytes.fromhex(k))
    return kY.format().hex() == C

def construct_token(C_: PublicKey, K: bytes, r: str):
    rK = PublicKey(K).multiply(r)
    # C = C_ - rK
    C = subtract_points(C_, rK)
    return C.format().hex()

def construct_inputs(blinded_sigs, rs, secrets):
    inputs = []
    for output, r, s in zip(blinded_sigs, rs, secrets):
        amount = output["amount"]
        # K is the public key for this token value
        K = bytes.fromhex(pubkeys.get(str(amount)))
        # C_ is blinded signature
        C_ = PublicKey(bytes.fromhex(output["C_"]))
        # C is unblinded signature
        C = construct_token(C_, K, r)
        assert verify_token(C, s.encode(), privkeys.get(str(amount)))
        inputs.append({
            "amount": amount,
            "C": C,
            "id": output["id"],
            "secret": s
        })
    return json.dumps(inputs)
    


In [100]:
outputs = blinded_sigs
construct_inputs(outputs, rs, secrets)


'[{"amount": 1, "C": "02114511fb00338ea95280db720c63ae119c68c03941df3a61223ed95e76ff9574", "id": "00b2f181c83b11aa", "secret": "1"}, {"amount": 2, "C": "02853da76d8dedbbcece373e312007cc3bb92711380c03a0ca96c56f427a0a50e0", "id": "00b2f181c83b11aa", "secret": "2"}, {"amount": 4, "C": "02e78ba0328f6947671a7964bbc1a433231f36c86090f8203ef9a6ea81869e8b6b", "id": "00b2f181c83b11aa", "secret": "4"}]'

In [101]:
!/usr/local/bin/lightning-cli --lightning-dir=/tmp/l1 -k cashu-melt quote=3694d3565dff167b inputs='[{"amount": 1, "C": "02114511fb00338ea95280db720c63ae119c68c03941df3a61223ed95e76ff9574", "id": "00b2f181c83b11aa", "secret": "1"}, {"amount": 2, "C": "02853da76d8dedbbcece373e312007cc3bb92711380c03a0ca96c56f427a0a50e0", "id": "00b2f181c83b11aa", "secret": "2"}, {"amount": 4, "C": "02e78ba0328f6947671a7964bbc1a433231f36c86090f8203ef9a6ea81869e8b6b", "id": "00b2f181c83b11aa", "secret": "4"}]'

{
   "paid": true,
   "preimage": "155ff532149397cd2f819adcb010360b04f99bfacc228d3150da3f549dd36798"
}


### 6. Check melt status

In [103]:
!$l1 cashu-check-melt 3694d3565dff167b

{
   "error": "quote not found"
}
